## Initial Feature Engineering 1

In [1]:
import numpy as np
import pandas as pd

order_new = pd.read_csv("orders_new.csv")
order_product_new = pd.read_csv("order_products_new.csv")
#order_new.drop('Unnamed: 0', axis = 1, inplace = True)
#order_new.drop('ranking', axis = 1, inplace = True)
#order_new.drop('count', axis = 1, inplace = True)

In [2]:
order_new = order_new.fillna(0)

In [3]:
# join orders_new and order_product_new together
join = pd.merge(order_product_new, order_new, on = 'order_id', how = 'left' )
join.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,eval_set
0,2,33120,1,0.0,202279,3,5,9,8.0,Prior
1,2,28985,2,0.0,202279,3,5,9,8.0,Prior
2,2,9327,3,0.0,202279,3,5,9,8.0,Prior
3,2,45918,4,0.0,202279,3,5,9,8.0,Prior
4,2,30035,5,0.0,202279,3,5,9,8.0,Prior


#### 1. order_product_prior

In [4]:
order_product_prior = join[join['eval_set'] == 'Prior']
order_product_prior.shape

(32434489, 10)

In [5]:
order_product_prior.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,eval_set
0,2,33120,1,0.0,202279,3,5,9,8.0,Prior
1,2,28985,2,0.0,202279,3,5,9,8.0,Prior
2,2,9327,3,0.0,202279,3,5,9,8.0,Prior
3,2,45918,4,0.0,202279,3,5,9,8.0,Prior
4,2,30035,5,0.0,202279,3,5,9,8.0,Prior


#### 2. user_feature_1

In [6]:
df1 = order_new[order_new['eval_set'] == 'Prior']

user_id = df1['user_id'].unique()
user_orders = df1.groupby(['user_id'])['order_number'].max()
sum_days_since_prior_order = df1.groupby(['user_id'])['days_since_prior_order'].sum()
avg_days_since_prior_order = df1.groupby(['user_id'])['days_since_prior_order'].mean()

In [7]:
user_feature_1 = pd.DataFrame({
    'user_orders': user_orders,
    'sum_days_since_prior_order': sum_days_since_prior_order,
    'avg_days_since_prior_order': avg_days_since_prior_order
})
user_feature_1

,user_orders,sum_days_since_prior_order,avg_days_since_prior_order
user_id,,,
1,10,176.0,17.600000
2,14,198.0,14.142857
3,12,133.0,11.083333
4,5,55.0,11.000000
5,4,40.0,10.000000
...,...,...,...
206205,3,40.0,13.333333
206206,67,249.0,3.716418
206207,16,215.0,13.437500


In [8]:
user_feature_1 = pd.DataFrame({
    'user_orders': user_orders,
    'sum_days_since_prior_order': sum_days_since_prior_order,
    'avg_days_since_prior_order': avg_days_since_prior_order
})
user_feature_1 = user_feature_1.reset_index()
user_feature_1.head(10)

,user_id,user_orders,sum_days_since_prior_order,avg_days_since_prior_order
0,1,10,176.0,17.600000
1,2,14,198.0,14.142857
2,3,12,133.0,11.083333
3,4,5,55.0,11.000000
4,5,4,40.0,10.000000
5,6,3,18.0,6.000000
6,7,20,203.0,10.150000
7,8,3,60.0,20.000000
8,9,3,36.0,12.000000
9,10,5,79.0,15.800000


#### 03. user_feature_2

In [9]:
df = order_product_prior[order_product_prior['order_number'] > 1]

total_products = order_product_prior.groupby('user_id')['product_id'].count()
distinct_products = order_product_prior.groupby('user_id')['product_id'].nunique()
reordered_sum = order_product_prior.groupby('user_id')['reordered'].sum()
reordered_count = df.groupby('user_id')['reordered'].count()

In [10]:
user_feature_2 = pd.DataFrame({
    'total_products': total_products,
    'distinct_products': distinct_products,
    'reordered_ratio': reordered_sum/reordered_count
})
user_feature_2.head(10)

,total_products,distinct_products,reordered_ratio
user_id,,,
1,59,18,0.759259
2,195,102,0.576923
3,88,33,0.666667
4,18,17,0.642857
5,37,23,0.961538
6,14,12,0.400000
7,206,68,0.680412
8,49,36,1.000000
9,76,58,1.000000


In [11]:
user_feature_2 = user_feature_2.reset_index()
user_feature_2.head(10)

,user_id,total_products,distinct_products,reordered_ratio
0,1,59,18,0.759259
1,2,195,102,0.576923
2,3,88,33,0.666667
3,4,18,17,0.642857
4,5,37,23,0.961538
5,6,14,12,0.400000
6,7,206,68,0.680412
7,8,49,36,1.000000
8,9,76,58,1.000000
9,10,143,94,0.239130


#### 04. up_feature

In [12]:
m = order_product_prior.groupby(['user_id','product_id'])

In [13]:
up_feature = pd.DataFrame({
    'up_orders': m['order_number'].count(),
    'up_first_order': m['order_number'].min(),
    'up_last_order': m['order_number'].max(),
    'up_average_cart_position': m['add_to_cart_order'].mean()
})

In [14]:
up_feature = up_feature.reset_index()
up_feature.head(10)

,user_id,product_id,up_orders,up_first_order,up_last_order,up_average_cart_position
0,1,196,10,1,10,1.400000
1,1,10258,9,2,10,3.333333
2,1,10326,1,5,5,5.000000
3,1,12427,10,1,10,3.300000
4,1,13032,3,2,10,6.333333
5,1,13176,2,2,5,6.000000
6,1,14084,1,1,1,2.000000
7,1,17122,1,5,5,6.000000
8,1,25133,8,3,10,4.000000
9,1,26088,2,1,2,4.500000


#### 5. prd_feature

In [15]:
order_product_prior['product_seq_time'] = order_product_prior.groupby(['user_id','product_id'])['order_number'].rank()

/Users/jinhou/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [16]:
order_product_prior

,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,eval_set,product_seq_time
0,2,33120,1,0.0,202279,3,5,9,8.0,Prior,2.0
1,2,28985,2,0.0,202279,3,5,9,8.0,Prior,3.0
2,2,9327,3,0.0,202279,3,5,9,8.0,Prior,1.0
3,2,45918,4,0.0,202279,3,5,9,8.0,Prior,2.0
4,2,30035,5,0.0,202279,3,5,9,8.0,Prior,1.0
...,...,...,...,...,...,...,...,...,...,...,...
32434484,3421083,39678,6,1.0,25247,24,2,6,21.0,Prior,3.0
32434485,3421083,11352,7,1.0,25247,24,2,6,21.0,Prior,1.0
32434486,3421083,4600,8,1.0,25247,24,2,6,21.0,Prior,1.0
32434487,3421083,24852,9,1.0,25247,24,2,6,21.0,Prior,12.0


In [17]:
m = order_product_prior.groupby('product_id')

In [18]:
order_product_prior[order_product_prior['product_seq_time'] == 2].groupby('product_id')['product_seq_time'].count()

product_id
1        276
2          8
3         36
4         64
5          4
        ... 
49684      1
49685      6
49686     16
49687      4
49688     11
Name: product_seq_time, Length: 45305, dtype: int64

In [19]:
order_product_prior[order_product_prior['product_seq_time'] == 1].groupby('product_id')['product_seq_time'].sum()

product_id
1        716.0
2         78.0
3         74.0
4        182.0
5          6.0
         ...  
49684      8.0
49685     43.0
49686     36.0
49687      7.0
49688     74.0
Name: product_seq_time, Length: 49677, dtype: float64

In [20]:
m['order_number'].count()

product_id
1        1852
2          90
3         277
4         329
5          15
         ... 
49684       9
49685      49
49686     120
49687      13
49688      89
Name: order_number, Length: 49677, dtype: int64

In [21]:
prd_feature = pd.DataFrame({
    'prod_orders': m['order_number'].count(),
    'prod_reorders': m['reordered'].sum(),
    'prod_first_orders': order_product_prior[order_product_prior['product_seq_time'] == 1].groupby('product_id')['product_seq_time'].sum(),
    'prod_second_orders': order_product_prior[order_product_prior['product_seq_time'] == 2].groupby('product_id')['product_seq_time'].count()
})

In [22]:
prd_feature = prd_feature.reset_index()
prd_feature.head(10)

,product_id,prod_orders,prod_reorders,prod_first_orders,prod_second_orders
0,1,1852,1066.0,716.0,276.0
1,2,90,51.0,78.0,8.0
2,3,277,167.0,74.0,36.0
3,4,329,192.0,182.0,64.0
4,5,15,8.0,6.0,4.0
5,6,8,3.0,5.0,2.0
6,7,30,22.0,18.0,6.0
7,8,165,88.0,82.0,30.0
8,9,156,91.0,74.0,31.0
9,10,2572,1481.0,1268.0,399.0


In [23]:
prd_feature

,product_id,prod_orders,prod_reorders,prod_first_orders,prod_second_orders
0,1,1852,1066.0,716.0,276.0
1,2,90,51.0,78.0,8.0
2,3,277,167.0,74.0,36.0
3,4,329,192.0,182.0,64.0
4,5,15,8.0,6.0,4.0
...,...,...,...,...,...
49672,49684,9,6.0,8.0,1.0
49673,49685,49,29.0,43.0,6.0
49674,49686,120,63.0,36.0,16.0
49675,49687,13,8.0,7.0,4.0


In [24]:
up_feature

,user_id,product_id,up_orders,up_first_order,up_last_order,up_average_cart_position
0,1,196,10,1,10,1.400000
1,1,10258,9,2,10,3.333333
2,1,10326,1,5,5,5.000000
3,1,12427,10,1,10,3.300000
4,1,13032,3,2,10,6.333333
...,...,...,...,...,...,...
13307948,206209,43961,3,4,12,8.000000
13307949,206209,44325,1,7,7,8.000000
13307950,206209,48370,1,11,11,8.000000
13307951,206209,48697,1,7,7,6.000000


## Initial Feature Engineering 2

In [25]:
data = pd.merge(user_feature_1, user_feature_2, on = 'user_id', how = 'left')
data = pd.merge(data, up_feature, on = 'user_id', how = 'left')
data = pd.merge(data, prd_feature, on = 'product_id', how = 'left')

In [26]:
data.head(10)

,user_id,user_orders,sum_days_since_prior_order,avg_days_since_prior_order,total_products,distinct_products,reordered_ratio,product_id,up_orders,up_first_order,up_last_order,up_average_cart_position,prod_orders,prod_reorders,prod_first_orders,prod_second_orders
0,1,10,176.0,17.6,59,18,0.759259,196,10,1,10,1.400000,35791,20502.0,8000.0,4660.0
1,1,10,176.0,17.6,59,18,0.759259,10258,9,2,10,3.333333,1946,1092.0,557.0,308.0
2,1,10,176.0,17.6,59,18,0.759259,10326,1,5,5,5.000000,5526,3114.0,1923.0,1003.0
3,1,10,176.0,17.6,59,18,0.759259,12427,10,1,10,3.300000,6476,3662.0,1679.0,889.0
4,1,10,176.0,17.6,59,18,0.759259,13032,3,2,10,6.333333,3751,2149.0,1286.0,617.0
5,1,10,176.0,17.6,59,18,0.759259,13176,2,2,5,6.000000,379450,217064.0,63537.0,45231.0
6,1,10,176.0,17.6,59,18,0.759259,14084,1,1,1,2.000000,15935,9029.0,3012.0,1895.0
7,1,10,176.0,17.6,59,18,0.759259,17122,1,5,5,6.000000,13880,8097.0,4503.0,2375.0
8,1,10,176.0,17.6,59,18,0.759259,25133,8,3,10,4.000000,6196,3509.0,1610.0,935.0
9,1,10,176.0,17.6,59,18,0.759259,26088,2,1,2,4.500000,2523,1417.0,1163.0,471.0


In [27]:
data['prod_reorder_probability'] = data.prod_second_orders / data.prod_first_orders
data['prod_reorder_times'] = 1 + data.prod_reorders / data.prod_first_orders
data['prod_reorder_ratio'] = data.prod_reorders / data.prod_orders
data.drop(['prod_reorders', 'prod_first_orders', 'prod_second_orders'], axis=1, inplace=True)
data['user_average_basket'] = data.total_products / data.user_orders
data['up_order_rate'] = data.up_orders / data.user_orders
data['up_orders_since_last_order'] = data.user_orders - data.up_last_order
data['up_order_rate_since_first_order'] = data.up_orders / (data.user_orders - data.up_first_order + 1)

In [28]:
prd = data['product_id']
data.drop(labels=['product_id'], axis=1,inplace = True)
data.insert(1, 'product_id', prd)
data.head(10)

,user_id,product_id,user_orders,sum_days_since_prior_order,avg_days_since_prior_order,total_products,distinct_products,reordered_ratio,up_orders,up_first_order,up_last_order,up_average_cart_position,prod_orders,prod_reorder_probability,prod_reorder_times,prod_reorder_ratio,user_average_basket,up_order_rate,up_orders_since_last_order,up_order_rate_since_first_order
0,1,196,10,176.0,17.6,59,18,0.759259,10,1,10,1.400000,35791,0.582500,3.562750,0.572826,5.9,1.0,0,1.000000
1,1,10258,10,176.0,17.6,59,18,0.759259,9,2,10,3.333333,1946,0.552962,2.960503,0.561151,5.9,0.9,0,1.000000
2,1,10326,10,176.0,17.6,59,18,0.759259,1,5,5,5.000000,5526,0.521581,2.619345,0.563518,5.9,0.1,5,0.166667
3,1,12427,10,176.0,17.6,59,18,0.759259,10,1,10,3.300000,6476,0.529482,3.181060,0.565473,5.9,1.0,0,1.000000
4,1,13032,10,176.0,17.6,59,18,0.759259,3,2,10,6.333333,3751,0.479782,2.671073,0.572914,5.9,0.3,0,0.333333
5,1,13176,10,176.0,17.6,59,18,0.759259,2,2,5,6.000000,379450,0.711884,4.416340,0.572049,5.9,0.2,5,0.222222
6,1,14084,10,176.0,17.6,59,18,0.759259,1,1,1,2.000000,15935,0.629150,3.997676,0.566614,5.9,0.1,9,0.100000
7,1,17122,10,176.0,17.6,59,18,0.759259,1,5,5,6.000000,13880,0.527426,2.798135,0.583357,5.9,0.1,5,0.166667
8,1,25133,10,176.0,17.6,59,18,0.759259,8,3,10,4.000000,6196,0.580745,3.179503,0.566333,5.9,0.8,0,1.000000
9,1,26088,10,176.0,17.6,59,18,0.759259,2,1,2,4.500000,2523,0.404987,2.218401,0.561633,5.9,0.2,8,0.200000


In [43]:
order_new.head(10)

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,eval_set
0,2774568,3,13,5,15,11.0,Test
1,329954,4,6,3,12,30.0,Test
2,1528013,6,4,3,16,22.0,Test
3,1376945,11,8,6,11,8.0,Test
4,1356845,12,6,1,20,30.0,Test
5,2161313,15,23,1,9,7.0,Test
6,1416320,16,7,0,13,7.0,Test
7,1735923,19,10,6,17,8.0,Test
8,1980631,20,5,1,11,30.0,Test
9,139655,22,16,5,6,1.0,Test


In [31]:
order_new1 = order_new[['user_id','eval_set']]

In [ ]:
# join orders_new and order_product_new together
new_df = pd.merge(order_new1, data, on = 'user_id', how = 'left' )
new_df.head()

In [38]:
user_feature_1.shape

(206209, 4)

In [39]:
user_feature_2.shape

(206209, 4)

In [40]:
up_feature.shape

(13307953, 6)

In [41]:
prd_feature.shape

(49677, 5)